# Hyperparameter Tuning for Social Network Simulation

This notebook explores the impact of different hyperparameters on the simulation performance.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
from itertools import product
from src.environment.social_network_env import SocialNetworkEnv
from src.agents.dqn_agent import DQNAgent
from src.training.train import train_network
from src.training.evaluation import evaluate_network

## 1. Define Parameter Grid

In [ ]:
# Define parameter ranges to test
param_grid = {
    'learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'hidden_size': [64, 128],
    'epsilon_decay': [0.995, 0.999]
}

# Create all combinations
param_combinations = [dict(zip(param_grid.keys(), v)) 
                     for v in product(*param_grid.values())]

## 2. Parameter Search Function

In [ ]:
def evaluate_parameters(params, num_episodes=200):
    """Evaluate a set of parameters."""
    # Initialize environment
    env = SocialNetworkEnv()
    
    # Initialize agent with parameters
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    agent = DQNAgent(
        state_dim,
        action_dim,
        learning_rate=params['learning_rate'],
        batch_size=params['batch_size'],
        hidden_size=params['hidden_size'],
        epsilon_decay=params['epsilon_decay']
    )
    
    # Train and evaluate
    training_stats = train_network(env, agent, num_episodes=num_episodes)
    eval_metrics = evaluate_network(env, agent)
    
    return {
        'params': params,
        'final_reward': training_stats['episode_rewards'][-1],
        'avg_reward': np.mean(training_stats['episode_rewards']),
        'eval_metrics': eval_metrics
    }

## 3. Run Parameter Search

In [ ]:
# Evaluate all parameter combinations
results = []
for params in param_combinations:
    print(f"Testing parameters: {params}")
    result = evaluate_parameters(params)
    results.append(result)
    
# Sort results by average reward
results.sort(key=lambda x: x['avg_reward'], reverse=True)

## 4. Analyze Results

In [ ]:
# Display top performing parameters
print("Top 3 Parameter Combinations:")
for i, result in enumerate(results[:3]):
    print(f"\n{i+1}. Parameters:")
    for k, v in result['params'].items():
        print(f"  {k}: {v}")
    print(f"  Average Reward: {result['avg_reward']:.2f}")
    print(f"  Final Reward: {result['final_reward']:.2f}")